<a href="https://colab.research.google.com/github/zrghassabi/VisionTransformers/blob/main/VisionTransformersBasics1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Certainly! Here's a basic illustration of the Vision Transformer (ViT) architecture with corresponding code snippets in PyTorch. The Vision Transformer processes images by dividing them into patches, applying attention mechanisms, and then using feed-forward networks for feature extraction.

Overview of Vision Transformer Structure
#Patch Embedding:
 Divide the image into fixed-size patches and embed each patch into a vector.
#Positional Encoding:
Add positional information to the patch embeddings.
#Transformer Encoder Layers:
Apply multiple layers of the transformer encoder, including multi-head self-attention and feed-forward networks.
#Classification Head:
 Apply a final layer to the output of the transformer encoder to produce the classification output.

In [ ]:
import torch
import torch.nn as nn

class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, num_classes=1000, hidden_dim=768, num_heads=12, num_layers=12):
        super(VisionTransformer, self).__init__()

        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2
        self.patch_dim = patch_size * patch_size * in_channels

        # Patch Embedding Layer
        self.patch_embedding = nn.Linear(self.patch_dim, hidden_dim)

        # Positional Encoding
        self.position_embedding = nn.Parameter(torch.zeros(1, self.num_patches + 1, hidden_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, hidden_dim))

        # Transformer Encoder Layers
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Classification Head
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        # x: [batch_size, channels, height, width]
        batch_size = x.size(0)

        # Create patches
        x = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        x = x.contiguous().view(batch_size, self.patch_size * self.patch_size * x.size(1), -1)
        x = x.permute(0, 2, 1)  # [batch_size, num_patches, patch_dim]

        # Patch embedding
        x = self.patch_embedding(x)

        # Add class token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)  # [batch_size, num_patches + 1, hidden_dim]

        # Add positional encoding
        x = x + self.position_embedding

        # Transformer Encoder
        x = x.permute(1, 0, 2)  # [num_patches + 1, batch_size, hidden_dim]
        x = self.transformer_encoder(x)
        x = x[0]  # Take the output of the class token

        # Classification head
        x = self.fc(x)

        return x

# Example usage
model = VisionTransformer(img_size=224, patch_size=16, num_classes=1000)
input_tensor = torch.randn(8, 3, 224, 224)  # Example batch of images
output = model(input_tensor)
print(output.shape)  # Output shape should be [8, 1000] for 8 images and 1000 classes


Explanation
#Patch Embedding:
The image is divided into patches of size patch_size x patch_size, and each patch is flattened and linearly embedded into a vector of size hidden_dim.

#Positional Encoding:
 Positional information is added to the patch embeddings to retain spatial information.

#Transformer Encoder:
 Multiple layers of the transformer encoder are applied, which include multi-head self-attention and feed-forward layers.

#Classification Head:
The output corresponding to the class token (the first token) is passed through a final linear layer to produce the class scores.

Certainly! To use the Vision Transformer (ViT) model for image classification, you'll need to follow these steps:

#Prepare the Dataset:
Load and preprocess your image data.
#Define the Model:
Use the Vision Transformer architecture.
#Train the Model:
Implement the training loop.
#Evaluate the Model:
 Test the model on validation or test data.

Here's a complete example using the Vision Transformer model for image classification in PyTorch. This example assumes you have a dataset and are familiar with basic PyTorch operations.

In [ ]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Set up data transformations and dataloaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Initialize the model, loss function, and optimizer
model = VisionTransformer(img_size=224, patch_size=16, num_classes=10)  # CIFAR-10 has 10 classes
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Evaluation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')